In [34]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mai 20

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pickle
import math
import random

# my own class files
import weight_normalise.Receptors as Receptors
import weight_normalise.Network as Network
import weight_normalise.create_update_synapse as cus


importlib.reload(Receptors)
importlib.reload(Network)
importlib.reload(cus)

path = "/home/yaning/Documents/Spiking_NN/"
np.set_printoptions(threshold=np.inf)

In [35]:
pointCount = 6000
# deltaTms = 0.05
# times = np.arange(pointCount) * deltaTms
# initial_Vm = 1.3458754117369027

Input neurons

In [36]:
# create input neurons 5x5 because after 2 conv2D and 2 pooling
# that's how many pixels left
input_size = 5

input_array = [[Network.Neuron((i, j)) for j in range(input_size)] for i in range(input_size)]
input_array_flat = list(itertools.chain.from_iterable(input_array))

In [37]:
input_array[3][4].id

(3, 4)

Excitatory Neurons

In [38]:
E_height = 10
E_width = 10

E_array = [[Network.Neuron((i, j)) for j in range(E_height)] for i in range(E_width)]
E_array_flat = list(itertools.chain.from_iterable(E_array))

In [39]:
E_array[3][7].id

(3, 7)

Inhibitory Neurons

In [40]:
I_amount = 10

I_array = [Network.Neuron((i)) for i in range(I_amount)]

Ouput Neurons

In [45]:
output_size = 5

output_array = [[Network.Neuron((i, j)) for j in range(output_size)] for i in range(output_size)]
output_array_flat = list(itertools.chain.from_iterable(output_array))

Connection them!

In [ ]:
# Input to E (all-to-all connection)
for i in input_array_flat:
    for j in E_array_flat:
        cus.create_synapse(i, j, "AMPA")

# E to I p=0.1
for i in E_array_flat:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "AMPA+NMDA")

# I to I p=0.1 (should i prevent self connection? they exist but rare)
for i in I_array:
    for j in I_array:
        if random.random() < 0.1:
            cus.create_synapse(i, j, "GABA")

# I to E p=0.024
for i in I_array:
    for j in E_array_flat:
        if random.random() < 0.024:
            cus.create_synapse(i, j, "GABA")

# # E to output (subsets)
stride = 2
kernel_size = 2

for i in output_array_flat:
    idx = i.id
    x_num = idx[0]
    y_num = idx[1]
    connect_arrary = []
    x_select = E_array[stride*x_num:stride*x_num+kernel_size]
    for j in range(len(x_select)):
        connect_arrary.append(x_select[j][stride*y_num:stride*y_num+kernel_size])
    
    connect_arrary = list(itertools.chain.from_iterable(connect_arrary))
    # connect all the selected E neurons
    for c in connect_arrary:
        cus.create_synapse(c, i, "AMPA")

Input neuron firing sequences

Run

2719